In [1]:
import copy
import time
from Model.data_model import get_data
from Model.data_model import parse_game
from Model.top_players import get_player_name, get_one_hot, player_list
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from bidict import bidict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sys
%load_ext autoreload
%autoreload 2

In [2]:
white_model = tf.keras.models.load_model('./white_player_model')
def guess(game):
    prediction_array = white_model.predict(game)
    print(type(prediction_array))
    index = np.argmax(prediction_array)
    print('Player is {}'.format(get_player_name(index)))


In [4]:
game = tf.constant([[[ 0, 14, 11,  0,  0, 21,  0, 24,],
 [ 0,  0, 15, 11,  0, 21, 23,  0,],
 [13,  0, 12,  0,  0,  0, 21,  0,],
 [ 0,  0, 13, 11, 21, 23, 22,  0,],
 [ 0,  0, 11, 22,  0,  0, 25,  0,],
 [14, 11, 12,  0,  0,  0, 21, 24,],
 [16, 11,  0,  0,  0,  0, 21, 26,],
 [ 0, 11,  0,  0,  0,  0, 21,  0,]]])
guess(game)


1/1 [==============================] - 0s 14ms/step
<class 'numpy.ndarray'>
Player is Alexander_Zubov


In [6]:
piece_to_int = {
        'p':21,
        'P':11,
        'n':22,
        'N':12,
        'b':23,
        'B':13,
        'r':24,
        'R':14,
        'q':25,
        'Q':15,
        'k':26,
        'K':16,
    }
int_to_piece = {
        21:'p',
        11:'P',
        22:'n',
        12:'N',
        23:'b',
        13:'B',
        24:'r',
        14:'R',
        25:'q',
        15:'Q',
        26:'k',
        16:'K',
    }

In [8]:
def numeric_to_fen(numeric_game):
    numpy_game = (numeric_game.numpy()[0]).T
    rows = list(reversed(np.array_split(numpy_game, indices_or_sections=8)))
    #Game is now seperated into rows starting at "Row 8", down to "Row 1" in chess notation
    fen = ""
    for row in rows:
        # Convert row to fen format
        space_counter = 0
        for item in row[0].tolist():
            if item == 0:
                space_counter += 1
            else:
                if space_counter != 0:
                    fen += str(space_counter)
                    space_counter = 0
                fen += int_to_piece[item]
        if space_counter != 0:
            fen += str(space_counter)
            space_counter = 0
        fen += "/"
    # Remove final slash on last line
    fen = fen[:-1]
    return fen

def fen_to_numeric(fen):
    fen_first_split = fen.split(sep='/')
    # Remove win loss info from the end of fen
    fen_first_split[7] = fen_first_split[7].split(sep = ' ')[0]
    game = []
    for input_row in reversed(fen_first_split):
        output_row = []
        for input_item in input_row:
            if input_item in piece_to_int:
                output_row.append(int(piece_to_int[input_item]))
            elif input_item.isnumeric():
                for i in range(int(input_item)):
                    output_row.append(0)
            else:
                assert false
        game.append(output_row)
    game = np.array(game).T
    game_tensor = tf.convert_to_tensor([game])
    return game_tensor
assert tf.math.reduce_all(tf.equal(game, fen_to_numeric(numeric_to_fen(game))))



In [17]:
def get_chess_guesses(fen):
    # Split into lines
    game = fen_to_numeric(fen)
    white_model = tf.keras.models.load_model('./white_player_model')
    black_model = tf.keras.models.load_model('./black_player_model')
    white_output = white_model.predict(game).ravel()
    black_output = black_model.predict(game).ravel()
    white_sum = sum(white_output)
    black_sum = sum(black_output)
    whiteGuesses = []
    blackGuesses = []
    for i in range(10):
        print(white_output.argmax())
        white_player = str(get_player_name(white_output.argmax()))
        black_player = str(get_player_name(black_output.argmax()))
        white_player_certainty = white_output[white_output.argmax()]/white_sum
        black_player_certainty = black_output[black_output.argmax()]/black_sum
        if white_player_certainty > 0:
            whiteGuesses.append([white_player, "{cert:.2f}%".format(cert=100*white_player_certainty)])
        if black_player_certainty > 0:
            blackGuesses.append([black_player, "{cert:.2f}%".format(cert=100*black_player_certainty)])
        white_output[white_output.argmax()] = -1.0
        black_output[black_output.argmax()] = -1.0
        print(white_player)
    return {'whiteGuesses': whiteGuesses, 'blackGuesses': blackGuesses}

def numeric_to_fen(numeric_game):
    numpy_game = (numeric_game.numpy()[0]).T
    rows = list(reversed(np.array_split(numpy_game, indices_or_sections=8)))
    #Game is now seperated into rows starting at "Row 8", down to "Row 1" in chess notation
    fen = ""
    for row in rows:
        # Convert row to fen format
        space_counter = 0
        for item in row[0].tolist():
            if item == 0:
                space_counter += 1
            else:
                if space_counter != 0:
                    fen += str(space_counter)
                    space_counter = 0
                fen += int_to_piece[item]
        if space_counter != 0:
            fen += str(space_counter)
            space_counter = 0
        fen += "/"
    # Remove final slash on last line
    fen = fen[:-1]
    return fen

def fen_to_numeric(fen):
    fen_first_split = fen.split(sep='/')
    # Remove win loss info from the end of fen
    fen_first_split[7] = fen_first_split[7].split(sep = ' ')[0]
    game = []
    for input_row in reversed(fen_first_split):
        output_row = []
        for input_item in input_row:
            if input_item in piece_to_int:
                output_row.append(int(piece_to_int[input_item]))
            elif input_item.isnumeric():
                for i in range(int(input_item)):
                    output_row.append(0)
            else:
                assert false
        game.append(output_row)
    game = np.array(game).T
    game_tensor = tf.convert_to_tensor([game])
    return game_tensor

In [18]:
fen = 'r4rk1/2pnqppp/pp1b4/3b4/1P1Pn3/PQ1BPN2/R4PPP/2B2RK1 w - - 0 14'

get_chess_guesses(fen)

1/1 [==============================] - 0s 48ms/step
22
Alexander_Zubov
11
Firouzja2003
2
Hikaru
1
DanielNaroditsky
27
wonderfultime
40
Salem-AR
25
Oleksandr_Bortnyk
19
Jospem
6
spicycaterpillar
7
Bigfish1995


{'whiteGuesses': [['Alexander_Zubov', '100.00%'],
  ['Firouzja2003', '0.00%'],
  ['Hikaru', '0.00%'],
  ['DanielNaroditsky', '0.00%'],
  ['wonderfultime', '0.00%'],
  ['Salem-AR', '0.00%'],
  ['Oleksandr_Bortnyk', '0.00%'],
  ['Jospem', '0.00%'],
  ['spicycaterpillar', '0.00%'],
  ['Bigfish1995', '0.00%']],
 'blackGuesses': [['howitzer14', '100.00%'],
  ['Jospem', '0.00%'],
  ['DanielNaroditsky', '0.00%'],
  ['wonderfultime', '0.00%'],
  ['Hikaru', '0.00%'],
  ['GM_dmitrij', '0.00%'],
  ['Firouzja2003', '0.00%'],
  ['Bigfish1995', '0.00%'],
  ['mishanick', '0.00%'],
  ['lachesisQ', '0.00%']]}